In [ ]:
import pandas as pd
import geopandas as gpd
import requests, json, geojson
from shapely.geometry import shape
import shapely.wkt
import os

import sys
sys.path.append('../scripts/')
from read_untils import read_file, create_folder, temp_record_query, temp_record_sdf

In [ ]:
from collections import defaultdict

### import data

- POA(shapefile)

In [ ]:
poa_sf = gpd.read_file('../data/tables/external_POA/POA_2021_AUST_GDA2020.shp')
poa_sf['geometry'] = poa_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
poa_gdf = gpd.GeoDataFrame(poa_sf)

In [ ]:
poa_gdf.head()

In [ ]:
poa_gdf.count()

In [ ]:
poa_gdf[poa_gdf.geometry.isna()]

In [ ]:
poa_gdf = poa_gdf.replace(to_replace='None', value=0).dropna()

In [ ]:
poa_gdf = poa_gdf[['POA_CODE21', 'geometry']]

- SA2(shapefile)

In [ ]:
sa2_sf = gpd.read_file('../data/tables/external_SA2/SA2_2021_AUST_GDA2020.shp')
sa2_sf['geometry'] = sa2_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sa2_sf = gpd.GeoDataFrame(sa2_sf)

In [ ]:
sa2_sf.head()

In [ ]:
sa2_sf = sa2_sf[['SA2_CODE21', 'geometry']]

- SA2 population data

In [ ]:
sa2_data = pd.read_csv('../data/tables/external_SA2_data.csv')

In [ ]:
sa2_dt_head = sa2_data.loc[:4]
sa2_dt = sa2_data.loc[5:]

In [ ]:
sa2_dt.head()

In [ ]:
sa2_dt = sa2_dt[list(sa2_dt.columns)[9:10] + list(sa2_dt.columns)[30:32]+list(sa2_dt.columns)[36:37]]
sa2_dt.head()

In [ ]:
sa2_dt.count()

In [ ]:
col_name1 = sa2_dt.iloc[:1].to_numpy().tolist()[0]
col_name2 = sa2_dt.iloc[1:2].to_numpy().tolist()[0]
origin_col = list(sa2_dt.columns)

In [ ]:
col_dict = defaultdict()
for i in range(4):
    if i == 0:
        col_dict[origin_col[i]] = col_name2[i]
    else:
        col_dict[origin_col[i]] = col_name1[i] + '(' + str(col_name2[i]) + ')'

In [ ]:
col_dict

In [ ]:
sa2_dt = sa2_dt.iloc[3:2457].reset_index(drop=True).rename(col_dict, axis=1)

In [ ]:
sa2_dt.head()

### SA2 geo join with SA2 data

In [ ]:
sa2_sf.head()

In [ ]:
sa2_dt.count()

In [ ]:
sa2_sf.count()

In [ ]:
sa2_sf = sa2_sf.dropna()
sa2_dt = sa2_dt.reset_index(drop=True)
sa2_sf.count()

In [ ]:
sa2 = pd.merge( sa2_sf, sa2_dt, left_on='SA2_CODE21', right_on='SA2 code', how='inner').drop('SA2 code', axis=1)

In [ ]:
sa2.head()

### geo intersection

In [ ]:
sa2['2020(no.)'] = sa2['2020(no.)'].astype('int')
sa2['2021(no.)'] = sa2['2021(no.)'].astype('int')
sa2.head()

In [ ]:
poa_gdf.head()

In [ ]:
sa2.iloc[0]['2020(no.)']

In [ ]:
def check_for_population(geom, sa2_gdf=sa2):
    total_area = geom.area
    sum_2020 = 0
    sum_2021 = 0
    for i in range(len(sa2)):
        if (total_area>0 and geom.intersects(sa2.iloc[i].geometry) == True):
            current_area = geom.intersection(sa2.iloc[i].geometry).area
            total_area -= current_area
            proportion = current_area / sa2.iloc[i].geometry.area
            sum_2020 += sa2.iloc[i]['2020(no.)']*proportion
            sum_2021 += sa2.iloc[i]['2021(no.)']*proportion
            # print(sum_2020, sum_2021, proportion, sa2.iloc[i]['2020(no.)'])
    return sum_2020, sum_2021


In [ ]:
poa_gdf['2020(no.)'] = None
poa_gdf['2021(no.)'] = None
for i in range(len(poa_gdf)):
    if(i%25==0):
        print(i)
    temp = poa_gdf.iloc[i]
    sum_2020, sum_2021 = check_for_population(temp.geometry)
    poa_gdf.iloc[i,2]=sum_2020
    poa_gdf.iloc[i,3]=sum_2021

In [ ]:
poa_gdf.head()

In [ ]:
poa_gdf = poa_gdf.drop('geometry', axis=1)

In [ ]:
poa_gdf.to_parquet('../data/curated/geo_pos_population.parquet', index=False)